In [5]:
import pandas as pd
import numpy as np

## 2 - Enrichissement du dictionnaire

In [2]:
#Importation du dictionnaire
cheminDico = r"C:\Users\yazid\Documents\Cnam\Annee3\Données non numérique - Nauge\cnam-athon-pronunciation-main\cnam-athon-pronunciation-main\data\dictionary\1757_Buchanan-J.csv" 
dfDicoBuch = pd.read_csv(cheminDico, delimiter=";", encoding="utf8")

# afficher à l'écran un échantillon aléatoire du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
9908,2,pēnal,L.,nbSyll>1 et pas d'accent tonic,NaN,a.,28.0,159
6119,2,hāte,NaN,nbSyll>1 et pas d'accent tonic,2{syll}. Contraire à sa règle de syllabation s...,v.,66.0,104
5322,2,fr_i_ĕʹndleſs,NaN,NaN,NaN,a.,53.0,93
14695,3,ŭnblooʹdy,NaN,NaN,NaN,a.,9.0,226
10519,4,prē-elĕʹcted,L.,NaN,NaN,a.,62.0,167


In [3]:
#Table mapping
cheminMap = r"C:\Users\yazid\Documents\Cnam\Annee3\Données non numérique - Nauge\cnam-athon-pronunciation-main\cnam-athon-pronunciation-main\data\dictionary\Buchanan_PronChar_counts.csv"
dfMap = pd.read_csv(cheminMap, delimiter=";", encoding="utf8")

# Affichage
dfMap.sample(n=5)

,char,occ,graph
54,),28,NaN
2,b,2288,b
4,c,5500,c
9,i,3783,i
58,í,2,i


In [6]:
#On remplace les vides par des chaines de caractères vides
dfMap = dfMap.replace(np.nan,'', regex = True)

In [7]:
#Changement et correction des mots
def correctionTexte(ancienMot,dfMap):
    Resultat = ""
    
    for lettre in ancienMot: 
        dfRowFinded = dfMap[dfMap['char'] == lettre]
        
        if len(dfRowFinded == 1):
            Resultat += dfRowFinded.iloc[0]['graph']
            
        else:
            Resultat += '?'
            
    return(Resultat)

In [8]:
#Ajout de la colonne avec les mots vedettes
dfDicoBuch['motVedette'] = dfDicoBuch.apply(lambda x: correctionTexte(x['pron'], dfMap),1)

### 2.1 - Création des colonnes Sampa et Ipa

In [9]:
#Importation de la base de données corrigée avec l'ensemble des données
CheminBigRichSample=r"C:\Users\yazid\Documents\Cnam\Annee3\Données non numérique - Nauge\cnam-athon-pronunciation-main\cnam-athon-pronunciation-main\data\dataframe\BigiRichSample.csv"
dfBRS = pd.read_csv(CheminBigRichSample, delimiter=";", encoding="utf8")

In [10]:
#On merge les deux tables et supprime la colonne en hwd qui correspond au mot vedette
DfDico = dfDicoBuch.merge(dfBRS, how='inner',left_on= 'motVedette', right_on='hwd')
del DfDico['hwd']
DfDico.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,pronSampa,ipa
7265,2,pŭʹrſ_e_r,NaN,NaN,NaN,n.,39.0,173,purser,p 3:r s 3:r,pɜ:rsɜ:r
6155,4,obnŏʹxĭ_o_us,L.,NaN,NaN,a.,3.0,150,obnoxious,A b n A k S @ s,ɑbnɑkʃəs
7410,2,rāʹv_e_n,NaN,NaN,NaN,n.,69.0,176,raven,r\ eI v @ n,ɹ?vən
1295,3,căʹſsĭā,L.,NaN,NaN,n.,27.0,46,cassia,k { S i: @,kæʃiːə
3290,2,explōʹde,L.,NaN,NaN,v.,38.0,86,explode,i k s p l @U d,iksploʊd


In [11]:
DfDico.to_csv(r"C:\Users\yazid\Documents\Cnam\Annee3\Données non numérique - Nauge\cnam-athon-pronunciation-main\cnam-athon-pronunciation-main\data\dataframe\BuchananRich.csv")

In [12]:
cheminBigi = r"C:\Users\yazid\Documents\Cnam\Annee3\Données non numérique - Nauge\cnam-athon-pronunciation-main\cnam-athon-pronunciation-main\data\dataframe\BuchananRich.csv"
Bigi = pd.read_csv(cheminBigi, delimiter=",", encoding="utf8")
Bigi.head(3)

,Unnamed: 0,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,pronSampa,ipa
0,0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs
1,1,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən
2,2,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd


In [13]:
dfBigi=Bigi[["motVedette","nbSyll","pron","pronSampa","ipa"]]
dfBigi.head(3)

,motVedette,nbSyll,pron,pronSampa,ipa
0,abacus,3,aʹbăcus,{ b @ k @ s,æbəkəs
1,abandon,3,ābăʹndon,@ b { n d @ n,əbændən
2,abandoned,4,ābăʹndŏned,@ b { n d @ n d,əbændənd


### 2.2 - Isoler les mots qui n'existent pas dans le Bigi

In [14]:
#On vérifie qu'il n'y a pas de données manquantes dans notre dataframe
dfBigi.isnull().any()

motVedette    False
nbSyll        False
pron          False
pronSampa     False
ipa           False
dtype: bool

In [15]:
dfBigi[dfBigi['motVedette'].isnull()]

,motVedette,nbSyll,pron,pronSampa,ipa
